# Gillespe ABM Explainer
This notebook explains an implementation of the Gillespe ABM in detail. The example used is a model of HIV and HCV transmission in a population of injecting drug users. The purpose of this explainer is to describe the archiatecture of the script can be understood and hopefully be adapted to other systems.

## What is a Gillespe ABM?
The Gillespe ABM is an agent based model (ABM) that uses the Gillespe algorithm for relatively fast computation. It is an ABM in the sense that each individual (agent) in the modelled population has their own identity and set of attributes. Agents interact with each other through network connections. 

It is a Gillespe algorithm in the sense that the system can be described a a set of poisson processes which occur at given rates. The algorithm sequentially produces events generated by these processes. The rate associated with each process may be affected by other events as they occur. The main difference between the Gillespe ABM and a normal Gillespe algorithm is that each individual in the population has their own set of processes. 

## Features of this model
The features included in this model are summarised in this diagram

<img src="compartmental.png" alt="drawing" width="500"/> 

There is a network agents distributed across 8 compartments. Compartments describe the disease status of the agent with respect to 2 diseases (2 HCV states S or I, and 4 HIV states S, A, C and R). Agents transition between compartments at given rates, some of these rates are fixed values, others change depending on the compartments of the agent's network neighbours. The are also initiations (represented by the diagonal arrows), and cessations (represented by the black circles).

## Initiation (creating new agents)
One of the features of this model is that new agents are be created throughout the duration of the simulation. We need to define how this works before we can move on, but if you are not interested in these details then you can move on to the next section.

Lets start with a simple example with two agents, 1 and 2 who share a network connection. The starting compartment of each node and the network neighbours of each node are stored as follows

In [1]:
N=2 # keep track of the number of nodes
agents=[1,2]
state_of={1:'SS', 2:'AI'}
neighbours_of={1:[2], 2:[1]}

We need to set some parameters. New agents enter the system with disease statuses given by the folowing probabilities 

In [2]:
HIV_status,background_HIV=['S','A','C','R'],[0.95,0.01,0.04,0]
HCV_status,background_HCV=['S','I'],[0.95,0.05]

They also immediately make a number of connections. This number drawn from a distribution with mean and dispersion parameters

In [3]:
mean_degree=4
degree_disp=1.5

We will define a function that creates a new agent. This requires taking the above information and updating it

In [4]:
import random
import numpy as np

def create_agent():
    global N
    # give them a name
    agent=N+1
    N=N+1
    
    # decide what state it is in when it enters
    HIV_state=random.choices(HIV_status,background_HIV)[0]
    HCV_state=random.choices(HCV_status,background_HCV)[0]
    trans_state=HIV_state+HCV_state

    # how many connections do they start with
    initial_degree=int(np.random.lognormal(np.log(mean_degree),np.log(degree_disp)))
 
    # link to some random people in the population
    neighbours_of[agent]=list(set(random.sample(agents,min(len(agents),initial_degree))))
    
    # contacts are non-directional so we create the reverse links
    for neighbour in neighbours_of[agent]:
        neighbours_of[neighbour].append(agent)

    # add to the list of agents
    agents.append(agent)

    # we have not yet assigned it to its state (we do that later) so put it in the "dead" state
    state_of[agent]='XX'

    # create the event that will put it into the correct state
    global event
    event=(agent,trans_state)

Lets see the effect of the "create agent" function

In [5]:
create_agent()
print('N=',N,'agents=',agents,'neighbours_of=',neighbours_of)

N= 3 agents= [1, 2, 3] neighbours_of= {1: [2, 3], 2: [1, 3], 3: [1, 2]}


That was just a demo. Let's go back to what we have before.

In [6]:
N=2 # keep track of the number of nodes
agents=[1,2]
state_of={1:'SS', 2:'AI'}
neighbours_of={1:[2], 2:[1]}

## Transition rates (the engine of the algorithm)
At the heart of the algorithm there is a record of rates for each process. This is kept as a dictionary

In [7]:
P={(1,'AS'):0.2,(1,'SI'):0.1,(1,'XX'):0.1,(2,'AS'):0.4,(2,'SI'):0.3,(2,'XX'):0.1}

Each dictionary entry has a tuple as key, the first entry is the name of the agent that the process is happening to, the second part is the state that the agent will transition into, 'XX' representes leaving the population. Rates can be 0 but its ok to simply not include those ones. We also have a rate of new agents being created

In [8]:
initiation_rate=0.5

To determine *when* the next event will happen we draw an exponentially distributed random variable with rate equal to the total rate of all possible events 

In [9]:
time=0 # set the time

# update the time
total_rate=sum(P.values())+initiation_rate
time=time+np.random.exponential(1/total_rate)

Determine *which* event will occur, we first ask if it is an initiation. If it is then we create a node, if not then we select from the transistions in P with probability proportional to the rates

In [10]:
# is it an initiation event?
if random.random()<random.choices([True, False],[initiation_rate,total_rate])[0]:
    create_agent()

# otherwise its a transition event between compartments
else:
    event=random.choices(list(P.keys()),list(P.values()))[0]


Remember that the "create agent" function generates an event. So whatever happens, at this stage we have an event to execute

In [11]:
print('event:',event)

event: (1, 'XX')


Lets unpack that

In [12]:
(agent, transition)=event
print('agent',agent,'will transition to',transition)

agent 1 will transition to XX


## Transitions between compartments
An important feature of the Gillespe algorithm is that when events occur (meaning when an agent moves compartment), some of the rates are adjusted. This always applies to the agent being moved - if it moves out of a compartment then the rate of it making the same move again must be 0 while other transition rates will be adjusted to reflect the compartment the agent is in. These "progression rates" are held in a dictionary as follows

In [13]:
progression_rates={
        # HCV reversion 
        ('SI','SS'):0.2, # HCV reversion is no HIV
        ('AI','AS'):0.1, # HCV reversion if HIV acute
        ('CI','CS'):0.06, # HCV reversion if HIV chronic
        ('RI','RS'):0.06, # HCV reversion if HIV ART enrolled
                 
        # HIV progression
        ('AS','CS'):4, # HIV progression
        ('AI','CI'):4, # HIV progression if HCV+
        
        # ART enrolment
        ('AS','RS'):0.0, # ART enrolment (acute and HepC-)
        ('AI','RI'):0.0, # ART enrolment (acute and HepC+)
        ('CS','RS'):0.0, # ART enrolment (chronic and HepC-)
        ('CI','RI'):0.0, # ART enrolment (chronic and HepC+)

        # cessation
        ('SS','XX'):0.15, # susceptible cessation or death
        ('SI','XX'):0.15, # HCV only cessation or death
        
        ('AS','XX'):0.15, # acute HIV only cessation or death
        ('AI','XX'):0.15, # acute HIV and HCV coinfection cessation or death
        
        ('CS','XX'):0.15, # cronic HIV only cessation or death
        ('CI','XX'):0.15, # cronic HIV and HCV coinfection cesation or death
        
        ('RS','XX'):0.15, # on ART only cessation or death
        ('RI','XX'):0.15 # on ART and HCV coinfection cesation or death
    }

In some cases it also applies to nodes connected to the agent in the network - if the agent moves into an infectious compartment then the rate of its neighbours moving to an infectious compartment increases. The amount of infection pressure that each node puts on another to transition to another state is held in the following dictionary

In [14]:
transmission_rates={
        # (state of node, transition state, state of neighbour) : infection rate per infector
    
        # HCV infection
        # if HIV negative
        ('SS','SI','SI'):0.06, # from HCV+ contacts that are HIV negative
        ('SS','SI','AI'):0.06, # from HCV+ contacts that are HIV acute
        ('SS','SI','CI'):0.06, # from HCV+ contacts that are HIV chronic
        ('SS','SI','RI'):0.06, # from HCV+ contacts that are ART enrolled

        ('SS','SI','SS'):0, # from HCV- contacts that are HIV negative
        ('SS','SI','AS'):0, # from HCV- contacts that are HIV acute
        ('SS','SI','CS'):0, # from HCV- contacts that are HIV chronic
        ('SS','SI','RS'):0, # from HCV- contacts that are ART enrolled        

        # if HIV acute
        ('AS','AI','SI'):0.3, # from HCV+ contacts that are HIV negative
        ('AS','AI','AI'):0.3, # from HCV+ contacts that are HIV acute
        ('AS','AI','CI'):0.3, # from HCV+ contacts that are HIV chronic
        ('AS','AI','RI'):0.3, # from HCV+ contacts that are ART enrolled

        ('AS','AI','SS'):0, # from HCV- contacts that are HIV negative
        ('AS','AI','AS'):0, # from HCV- contacts that are HIV acute
        ('AS','AI','CS'):0, # from HCV- contacts that are HIV chronic
        ('AS','AI','RS'):0, # from HCV- contacts that are ART enrolled
        
        # if HIV chronic            
        ('CS','CI','SI'):0.3, # from HCV+ contacts that are HIV negative
        ('CS','CI','AI'):0.3, # from HCV+ contacts that are HIV acute
        ('CS','CI','CI'):0.3, # from HCV+ contacts that are HIV chronic
        ('CS','CI','RI'):0.3, # from HCV+ contacts that are ART enrolled
        
        ('CS','CI','SS'):0, # from HCV- contacts that are HIV negative
        ('CS','CI','AS'):0, # from HCV- contacts that are HIV acute
        ('CS','CI','CS'):0, # from HCV- contacts that are HIV chronic
        ('CS','CI','RS'):0, # from HCV- contacts that are ART enrolled
        
        # if on ART
        ('RS','RI','SI'):0.3, # from HCV+ contacts that are HIV negative
        ('RS','RI','AI'):0.3, # from HCV+ contacts that are HIV acute
        ('RS','RI','CI'):0.3, # from HCV+ contacts that are HIV chronic
        ('RS','RI','RI'):0.3, # from HCV+ contacts that are ART enrolled
        
        ('RS','RI','SS'):0, # from HCV- contacts that are HIV negative
        ('RS','RI','AS'):0, # from HCV- contacts that are HIV acute
        ('RS','RI','CS'):0, # from HCV- contacts that are HIV chronic
        ('RS','RI','RS'):0, # from HCV- contacts that are ART enrolled
        
        # HIV infection
        
        # if HCV negative
        ('SS','AS','SS'):0, # from HIV susceptible contacts that are HCV negative
        ('SS','AS','SI'):0, # from HIV susceptible contacts that are HCV positive
        
        ('SS','AS','AS'):0.3, # from HIV acute contacts that are HCV negative
        ('SS','AS','AI'):0.3, # from HIV acute contacts that are HCV positive

        ('SS','AS','CS'):0.03, # from HIV chronic contacts that are HCV negative
        ('SS','AS','CI'):0.03, # from HIV chronic contacts that are HCV positive

        ('SS','AS','RS'):0.01, # from ART enrolled contacts that are HCV negative
        ('SS','AS','RI'):0.01, # from ART enrolled contacts that are HCV positive
        
        # if HCV positive
        ('SI','AI','SS'):0, # from HIV susceptible contacts that are HCV negative
        ('SI','AI','SI'):0, # from HIV susceptible contacts that are HCV positive
        
        ('SI','AI','AS'):0.3, # from HIV acute contacts that are HCV negative
        ('SI','AI','AI'):0.3, # from HIV acute contacts that are HCV positive

        ('SI','AI','CS'):0.03, # from HIV chronic contacts that are HCV negative
        ('SI','AI','CI'):0.03, # from HIV chronic contacts that are HCV positive

        ('SI','AI','RS'):0.01, # from ART enrolled contacts that are HCV negative
        ('SI','AI','RI'):0.01, # from ART enrolled contacts that are HCV positive  
    }

## Executing a transition
Earlier we chose an agent and decided which compartment to move it to. To execute this we break it down into three stages: first it leaves whatever compartment it is currently in, then we update the record of which compartment it is in, then we enter it into a new compartment.

Lets start by defining the "leave" function. It takes the current state of the node as an argument

In [15]:
def leave(state):

    # set ALL transistions for the node to 0
    for hiv_state in HIV_status:
        for hcv_state in HCV_status:
            P[(agent,hiv_state+hcv_state)]=0

    # and set the cessation rate to 0
    P[(agent,'XX')]=0
    
    # reduce HIV pressure on neighbours
    if state[0]=='A': # if leaving HIV acute state 
        
        # decrease the infection rate of the neighbours of the event node
        for neighbour in neighbours_of[agent]:
            # only if they are susceptible to HIV
            if state_of[neighbour][0]=='S':
                # trans_state is the state the neighbour can transition to
                trans_state='A'+state_of[neighbour][1]
                # decrease by the amount that the node was contributing to the infection pressure of neighbour
                P[(neighbour,trans_state)]=P[(neighbour,trans_state)]-transmission_rates[(state_of[neighbour],trans_state,state)]
               
    # reduce HIV pressure on neighbours
    if state[0]=='C': # if leaving HIV chronic state
        
        # decrease the infection rate of the neighbours of the event node
        for neighbour in neighbours_of[agent]:
            # only if they are susceptible to HIV
            if state_of[neighbour][0]=='S':
                # trans_state is the state the neighbour can transition to
                trans_state='A'+state_of[neighbour][1]
                # reduction as going from acute to chronic   
                P[(neighbour,trans_state)]=P[(neighbour,trans_state)]-transmission_rates[(state_of[neighbour],trans_state,state)]
             
    # reduce HCV pressure on neighbours    
    if state[1]=='I': # if leaving HCV chronic state
        
        # decrease the infection rate of the neighbours of the event node
        for neighbour in neighbours_of[agent]:
            # only if they are susceptible to HCV
            if state_of[neighbour][1]=='S':
                # trans_state is the state the neighbour can transition to
                trans_state=state_of[neighbour][0]+'I'
                # this bit of code is same as above. Can it be written more efficiently?
                P[(neighbour,trans_state)]=P[(neighbour,trans_state)]-transmission_rates[(state_of[neighbour],trans_state,state)]
                

This function acts on the the current state of the agent, so is executed as follows

In [16]:
leave(state_of[agent])

The second part of the transition is somewhat simpler. We change the compartment of the agent to that of the chosen transition

In [17]:
state_of[agent]=transition

print(state_of)

{1: 'XX', 2: 'AI'}


The third step is to execute the rate changes that occur as a result of entering the new compartment. There are severalparts to this: changes related HIV related transitions, HCV transitions, and leaving the network

In [18]:
def enter(state):
    
    # Entering HIV susceptible
    if state[0]=='S': # if entering a HIV susceptible state
        # define the compartment the agent can move into
        trans_state='A'+state[1]
        # sum all the infection pressure from all the neighbours
        infection_pressure=sum(transmission_rates[(state,trans_state,state_of[contact])] for contact in neighbours_of[agent])
        # this is the new rate for this transition
        P[(agent,trans_state)]=infection_pressure

    # Entering HIV acute
    if state[0]=='A': # if entering a HIV acute compartment
        # define the compartment the agent can move into
        trans_state='C'+state[1]
        # change the recovery rate of the node from 0 to whatever it is 
        P[(agent,trans_state)]=progression_rates[(state,trans_state)]

        # increase the infection rate of the neighbours of the event node
        for neighbour in neighbours_of[agent]:#+sexual_neighbours[node]:
            # only if they are susceptible to HIV
            if state_of[neighbour][0]=='S':
                # define the compartment the neighbour can move into
                trans_state='A'+state_of[neighbour][1]
                # increase by the amount that the node contributes to the infection pressure of neighbour
                P[(neighbour,trans_state)]=P[(neighbour,trans_state)]+transmission_rates[(state_of[neighbour],trans_state,state_of[agent])]

    # Entering HIV chronic
    elif state[0]=='C': # if entering a HIV chrocic compartment
        # define the compartment the agent can move into
        trans_state='R'+state[1]
        # change the recovery rate of the node from 0 to whatever it is 
        P[(agent,trans_state)]=progression_rates[(state,trans_state)]

        # increase the infection rate of the neighbours of the event node
        for neighbour in neighbours_of[agent]:
            # only if they are susceptible to HIV
            if state_of[neighbour][0]=='S':
                # define the compartment the neighbour can move into
                trans_state='A'+state_of[neighbour][1]
                # increase by the amount that the node contributes to the infection pressure of neighbour
                P[(neighbour,trans_state)]=P[(neighbour,trans_state)]+transmission_rates[(state_of[neighbour],trans_state,state_of[agent])]

    #~~~~#
    
    # Entering HCV susceptible
    if state[1]=='S': # if entering a HCV suscepptible compartment 
        # define the compartment the agent can move into
        trans_state=state[0]+'I'
        # sum all the infection pressure from all the neighbours
        infection_pressure=sum(transmission_rates[(state,trans_state,state_of[contact])] for contact in neighbours_of[agent])        
        # change the infection rate of the event node from 0 to whatever it is
        P[(agent,trans_state)]=infection_pressure
     
    # Entering HCV infected
    elif state[1]=='I': # if entering a HCV infected compartment 
        # define the compartment the agent can move into
        trans_state=state[0]+'S'
        # change the recovery rate of the node from 0 to whatever it is 
        P[(agent,trans_state)]=progression_rates[(state,trans_state)]

        # increase the infection rate of the neighbours of the event node
        for neighbour in neighbours_of[agent]:
            # only if they are susceptible to HCV
            if state_of[neighbour][1]=='S':
                # define the compartment the neighbour can move into
                trans_state=state_of[neighbour][0]+'I'
                # increase by the amount that the node contributes to the infection pressure of neighbour 
                P[(neighbour,trans_state)]=P[(neighbour,trans_state)]+transmission_rates[(state_of[neighbour],trans_state,state_of[agent])]

    #~~~~#                

    # whatever state they enter (except the "ceased" state), give them a cessation rate
    if state!='XX':
        # change the cessation rate of the node from 0 to whatever it is 
        P[(agent,'XX')]=progression_rates[(state,'XX')]
        
    # leaving the network (if state == 'XX')
    else:
    
        # destroy the agent and its links
        agents.remove(agent)
        # remove from other nodes' neighbour lists
        for neighbour in neighbours_of[agent]:
            neighbours_of[neighbour].remove(agent)
        # remove from dictionary
        neighbours_of.pop(agent)
        state_of.pop(agent)
               
        # Lastly, a bit of clean-up. loop over all trasitions relevant to the node 
        for hiv_state in HIV_status:
            for hcv_state in HCV_status:
                P.pop((agent,hiv_state+hcv_state))

        P.pop((agent,'XX'))

This function acts on the new comaprtment of the agent, which we have just updated, so we do

In [19]:
enter(state_of[agent])

## The loop
We have all the elemnts that go intothe algorithm, we just have to loop them together. In summary one iteration goes like this

1. Choose the time of the next event
2. Choose the event - which means an agent and a compartment to transition to
3. Execute all the rate changes that occur from the agent leaving their current compartment
4. Update their compartment
5. Execute all the rate changes that occur from the agent entering their new compartment

In [20]:
end_time=10 

while time<end_time and agents: # two termination criteria, time and running out of people
    # 1. Choose the time of the next event
    total_rate=sum(P.values())+initiation_rate
    time=time+np.random.exponential(1/total_rate)
    
    # 2. Choose the event
    # is it an initiation event?
    if random.random()<random.choices([True, False],[initiation_rate,total_rate])[0]:
        create_agent()
    # otherwise its a transition event between compartments
    else:
        event=random.choices(list(P.keys()),list(P.values()))[0]
    # unpack
    (agent, transition)=event
    print('At time '+str(round(time,2))+', agent '+str(agent)+' transitioned from '+state_of[agent]+' to '+transition)
          
    # 3. Execute all the rate changes that occur from the agent leaving their current compartment
    leave(state_of[agent])

    # 4. Update their compartment
    state_of[agent]=transition
   
    # 5. Execute all the rate changes that occur from the agent entering their new compartment
    enter(state_of[agent])
 

At time 1.17, agent 2 transitioned from AI to AS
At time 1.29, agent 2 transitioned from AS to CS
At time 3.29, agent 2 transitioned from CS to XX


So now we have a simulation of two bloodborne diseases spreading on networked dynamic population! 

There are some features that I simplified for the sake of making it more readable, but the key elements are here. The more "operational" versions I use for analyisis have a bunch of extra features (like making more interesting network structures, various simulated interventions) and some tricks to speed them up.